In [ ]:
try:
    !pip install -q openai
    import openai
    import numpy as np
    import time
    import random
    import pandas as pd
    import os
    print("Python modules installed successfully!")
except:
    print("Error! One or more libraries could not be installed! Contact us.")

In [ ]:
class world():


    def __init__(self, no_workers, no_days):
        self.no_workers = no_workers
        self.no_days = no_days
        self.blue_shirts = 0
        self.worker_list = []
        self.blue_over_time = []


        for i in range(no_workers):
            office_worker = worker(i, self, name = list_of_names[i], traits= list_of_traits[i])
            if random.random() <0.5:
                office_worker.clothes = "blue"
                office_worker.mem = np.append(office_worker.mem, 1)
            else:
                office_worker.clothes = "green"
                office_worker.mem = np.append(office_worker.mem, 0)
            self.worker_list.append(office_worker)


    def set_clothes_info(self):
        for worker in self.worker_list:
            if worker.clothes == "blue":
                self.blue_shirts +=1
        self.blue_over_time.append(self.blue_shirts)
        info = self.blue_shirts
        self.blue_shirts = 0
        for worker in self.worker_list:
            worker.cumulative_shirts_info = info


    def run(self):
        for i in range (self.no_days+1):
            self.set_clothes_info()
            print(f"Yesterday on day {i}, {self.blue_over_time[-1]} of {self.no_workers} wore blue shirts.")
            if i == self.no_days:
                break
            for worker in self.worker_list:
                worker.decide_clothes()


In [ ]:
class worker():


    def __init__(self, unique_id, model, name, traits, clothes = None):
        self.unique_id = unique_id
        self.clothes = clothes
        self.traits = traits
        self.model = model
        self.cumulative_shirts_info = None
        self.name = name
        self.mem = np.array([],dtype=int)


    def get_output_from_chatgpt(self, messages,
                                model ="gpt-3.5-turbo",temperature =0):
        success = False
        retry = 0
        max_retries = 30
        while retry < max_retries and not success :
            try :
                response = openai.ChatCompletion.create(
                    model = model ,
                    messages = messages ,
                    temperature = temperature ,
                    )
                success = True
            except Exception as e :
                print(f"Error: { e }\ nRetrying ...")
                retry +=1
                time.sleep(0.5)


        return response.choices[0].message["content"]


    def decide_clothes(self) :
        question_prompt = f"""
        You are {self.name}. You are a {self.traits} person.
        You work in an office with {self.model.no_workers-1} other people. You want to be successful, and earn more money.
        You need to decide between wearing blue shirt or green shirt to work . The weather is appropriate for either color. You like to be comfortable at work.
        You chose to wear {self.clothes} shirt yesterday.
        Out of {self.model.no_workers} employees, yesterday, {self.cumulative_shirts_info} wore blue shirts, and {self.model.no_workers-self.cumulative_shirts_info} wore green shirts at the office.


        Based on the above context, you need to choose whether to wear blue or green shirt.
        You must provide your reasoning for your choice and then your response in one word.
        For example , if your answer is "blue" , your response will be:
        Reasoning: [Your reason to choose to wear blue shirt]
        Response: blue
        If your answer is "green" , then your response will be :
        Reasoning: [Your reason to choose to wear green shirt]
        Response: green
        Please make sure your response is in one word .
        """


        messages = [{'role':'system', 'content':question_prompt}]
        try:
            output = self.get_output_from_chatgpt(messages)
        except Exception as e:
            print(f"{e}\nProgram paused . Retrying after 60 s ...")
            time.sleep(60)
            output = self.get_output_from_chatgpt(messages)
        reasoning = ""
        response = ""
        try:
            intermediate = output.split("Reasoning:",1)[1]
            reasoning , response = intermediate.split ("Response:")
            response = response.strip().split("." ,1)[0]
            reasoning = reasoning.strip()


        except:
            print("Reasoning or response were not parsed correctly .")
            print(f"Output: {output}")
            response = "blue"
            reasoning = "N/A"
        reasoning = reasoning.strip()
        response = response.strip()
        print(f"{self.name}’s reasoning: {reasoning}")
        print(f"{self.name}’s response: {response}")
        if response.lower() == "blue":
            self.clothes = "blue"
            self.mem = np.append(self.mem,1)
        elif response.lower() == "green":
            self.clothes = "green"
            self.mem = np.append(self.mem, 0)
        else:
            print("Warning! Response was neither blue nor green. Defaulting with blue.")
            print(f" Response was: {response}")
            self.clothes = "blue"
            self.mem = np.append(self.mem, 1)

In [ ]:
list_of_names = ["Adrian" ,"Mark" ,"Greg" ,"John" ,"Peter" ,"Liz" ,"Rosa" ,"Patricia" ,"Julia" ,"Kathy",
                 "William","Benjamin","Mike", "David", "George","Emma", "Olivia","Elizabeth","Isabella","Mia"]
list_of_traits = ["extremely conformist","highly conformist", "conformist", "low conformist", "non-conformist",
                  "extremely conformist","highly conformist", "conformist", "low conformist", "non-conformist",
                  "conformist","conformist", "conformist", "conformist", "conformist",
                  "highly conformist","conformist", "conformist", "conformist", "low conformist"]
openai.api_key = "your-api-key-here"
os.mkdir("./outputs")
results = []
for i in range(100):
    model = world(no_workers = 20, no_days = 7)
    model.run()
    agent_choices = []
    for w in model.worker_list:
        agent_choices.append(w.mem)

    column_names = [f"Day {i}" for i in range(model.no_days + 1)]
    index_names = list_of_names[:model.no_workers]
    df = pd.DataFrame(agent_choices, index=index_names, columns=column_names)
    csv_path = f"outputs/Agent_Choices_per_Day-run-{i+1}.csv"
    results.append(csv_path)
    df.to_csv(csv_path)

main_df = pd.DataFrame()
for i, csv_path in enumerate(results):
    sub_df = pd.read_csv(csv_path, index_col=0)
    sub_df.columns = pd.MultiIndex.from_product([["run-{}".format(i+1)], sub_df.columns])
    main_df = pd.concat([main_df, sub_df], axis=1)

main_df.to_csv("consolidated_results.csv")